In [55]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns

#random seed dùng trong  np.random.seed(xxxxxx) là  6 chữ số cuối cùng trong mã số học viên cao học.
np.random.seed(101024)
sns.set_theme(style='whitegrid')

## Define problems

In [2]:
def sphere_function(x):
    return sum([(xi ** 2) for xi in x])

def griewank_function(x):
    sum_sq = sum([(xi ** 2) / 4000 for xi in x])
    prod_cos = math.prod([math.cos(xi / math.sqrt(i + 1)) for i, xi in enumerate(x)])
    return sum_sq - prod_cos + 1

def rosenbrock_function(x):
    sum_term = sum([100 * (x[i + 1] - x[i] ** 2) ** 2 + (x[i] - 1) ** 2 for i in range(len(x) - 1)])
    return sum_term

def michalewicz_function(x):
    m = 10
    return -sum([math.sin(xi) * (math.sin((i + 1) * xi ** 2 / math.pi) ** (2 * m)) for i, xi in enumerate(x)])

## Differential Evolution

In [56]:

def differential_evolution(target_func, bounds, pop_size=50, max_iter=1000, F=0.5, CR=0.9):
    # Initialize population within bounds
    pop = np.random.uniform(bounds[0], bounds[1], size=(pop_size, len(bounds[0])))
    
    for i in range(max_iter):
        for j in range(pop_size):
            # Select three random individuals
            candidates = np.random.choice(pop_size, 3, replace=False)
            
            # Mutation
            mutant = pop[candidates[0]] + F * (pop[candidates[1]] - pop[candidates[2]])
            
            # Crossover
            trial = np.copy(pop[j])
            for k in range(len(bounds[0])):
                if np.random.rand() > CR:
                    trial[k] = mutant[k]
                    
            # Clip trial within bounds
            trial = np.clip(trial, bounds[0], bounds[1])
            
            # Evaluate fitness
            f_trial = target_func(trial)
            f_target = target_func(pop[j])
            
            # Select based on fitness
            if f_trial <= f_target:
                pop[j] = trial
    
    # Return the best solution found
    best_index = np.argmin([target_func(ind) for ind in pop])
    return pop[best_index], target_func(pop[best_index]sphere_function)

# Usage example for Sphere function
bounds_sphere = ([-5.12] * 5, [5.12] * 5)  # Example bounds for Sphere function
best_solution_sphere, best_fitness_sphere = differential_evolution(sphere_function, bounds_sphere)
print("Best Solution (Sphere):", best_solution_sphere)
print("Best Fitness (Sphere):", best_fitness_sphere)

Best Solution (Sphere): [0.98671891 0.97306244 0.95559071 0.92260995 0.85803666]
Best Fitness (Sphere): 0.030136747606289937


## Evolution strategy

In [4]:

def evolution_strategy(target_func, bounds, mu=50, lambda_=200, max_iter=1000, sigma=0.1):
    # Initialize mean and covariance matrix
    mean = np.random.uniform(bounds[0], bounds[1], size=len(bounds[0]))
    cov = np.identity(len(bounds[0])) * sigma
    
    for i in range(max_iter):
        # Generate offspring
        offspring = np.random.multivariate_normal(mean, cov, lambda_)
        
        # Clip offspring within bounds
        offspring = np.clip(offspring, bounds[0], bounds[1])
        
        # Evaluate fitness
        fitness = [target_func(ind) for ind in offspring]
        
        # Sort by fitness
        sorted_indices = np.argsort(fitness)
        offspring = offspring[sorted_indices]
        
        # Update mean using mu best individuals
        mean = np.mean(offspring[:mu], axis=0)
    
    # Return the best solution found
    return mean, target_func(mean)

# Usage example for Griewank function
bounds_griewank = ([-600] * 10, [600] * 10)  # Example bounds for Griewank function
best_solution_griewank, best_fitness_griewank = evolution_strategy(griewank_function, bounds_griewank)
print("Best Solution (Griewank):", best_solution_griewank)
print("Best Fitness (Griewank):", best_fitness_griewank)

Best Solution (Griewank): [ 310.50503628   97.50058109  249.33936693 -175.12093233  202.42942231
  282.92385623  -74.24820968  123.2363285   261.6359895   -98.40514177]
Best Fitness (Griewank): 105.39407944547709


## Run

In [ ]:
n_trials = 10
my_id = 101024
for i in range(n_trials):
    np.random.seed(my_id + i)